In [1]:
# Importing Basic libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import seaborn as sns

from collections import Counter
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, plot_confusion_matrix, roc_auc_score,f1_score


from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

C:\Users\LENOVO\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# Libraries for text preprocessing
import re
import nltk

#nltk.download('stopwords')
from nltk.corpus import stopwords # import stopwords 

#nltk.download('wordnet') 
from nltk.stem.wordnet import WordNetLemmatizer # to lemmatize the words

from nltk.tokenize import RegexpTokenizer, word_tokenize # to tokenize sentences into words

# Loading Train dataset

In [4]:
data = pd.read_csv("C:\\Users\\LENOVO\\Desktop\\train.csv")

In [5]:
data.shape
data.head(3) # (14343, 3)

ID                                             Review  Rating
0   0  exceptional service nice all-around daughter s...       5
1   1  beautiful relaxing jw marriott desert ridge re...       5
2   2  great location great location 5 mins subway ta...       5

# Pre-processing Train Dataset

In [6]:
# Replacing contractions (shorter vesions of words) with full word (n't , 'nt  as not)
for index, row in data.iterrows():
    phrase = row['Review']
    phrase = phrase.lower() #Convert to lowercase
    phrase = re.sub(r"n\'t", " not", phrase) # replace n't  with not
    phrase = re.sub(r"\'nt", " not", phrase) # replace 'nt  with not 
    data.at[index, 'Review_2'] = phrase
data[0:3]

ID                                             Review  Rating  \
0   0  exceptional service nice all-around daughter s...       5   
1   1  beautiful relaxing jw marriott desert ridge re...       5   
2   2  great location great location 5 mins subway ta...       5   

                                            Review_2  
0  exceptional service nice all-around daughter s...  
1  beautiful relaxing jw marriott desert ridge re...  
2  great location great location 5 mins subway ta...

In [7]:
# TEXT NORMALIZATION   or PRE-PROCESSING

##Creating a list of stop words and adding custom stopwords
stop_words = set(stopwords.words("english")) -set(["not"])
##Creating a list of custom stopwords
new_words = ['n',"th","hotel",'hotels','room','rooms','restaurant','restaurants','e','ca','nd','wo','el','etc']
stop_words = stop_words.union(new_words)

for index, row in data.iterrows():
    text = re.sub('[^a-zA-Z]', ' ', row['Review_2']) # Remove punctuations    
    # text = text.lower() #Convert to lowercase
    text = re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)  # remove tags
    text = re.sub("(\\d|\\W)+"," ",text)  # remove special characters and digits
    text = text.split()  # Convert to list from string
    
    #Lemmatisation
    lemmatizer = WordNetLemmatizer()
    text = [lemmatizer.lemmatize(word) for word in text if not word  in stop_words] 
    text_string = " ".join(text)
    data.at[index, 'Clean_review'] = text_string
data.head(3)

ID                                             Review  Rating  \
0   0  exceptional service nice all-around daughter s...       5   
1   1  beautiful relaxing jw marriott desert ridge re...       5   
2   2  great location great location 5 mins subway ta...       5   

                                            Review_2  \
0  exceptional service nice all-around daughter s...   
1  beautiful relaxing jw marriott desert ridge re...   
2  great location great location 5 mins subway ta...   

                                        Clean_review  
0  exceptional service nice around daughter staye...  
1  beautiful relaxing jw marriott desert ridge re...  
2  great location great location min subway take ...

In [8]:
data.drop(['Review_2'], axis=1,inplace=True)

In [19]:
data['tokens'] = data["Clean_review"].apply(word_tokenize)

In [20]:
data.head()

ID                                             Review  Rating  \
0   0  exceptional service nice all-around daughter s...       5   
1   1  beautiful relaxing jw marriott desert ridge re...       5   
2   2  great location great location 5 mins subway ta...       5   
3   3  pleased nice safe hotel, flower market hotel v...       3   
4   4  excellent hotel service great hotel excellent ...       4   

                                        Clean_review  \
0  exceptional service nice around daughter staye...   
1  beautiful relaxing jw marriott desert ridge re...   
2  great location great location min subway take ...   
3  pleased nice safe flower market vast array mai...   
4  excellent service great excellent location cou...   

                                              tokens  
0  [exceptional, service, nice, around, daughter,...  
1  [beautiful, relaxing, jw, marriott, desert, ri...  
2  [great, location, great, location, min, subway...  
3  [pleased, nice, safe, flower, market, vast, ar...  
4  [excellent, service, great, excellent, locatio...

# Word2vec for Train Dataset

In [21]:
from gensim.models import Word2Vec
import time

size = 100
window = 3
min_count = 1
workers = 3
sg = 1

OUTPUT_FOLDER=""

word2vec_model_file1 = OUTPUT_FOLDER + 'word2vec_' + str(size) + '.model'
start_time = time.time()
stemmed_tokens = pd.Series(data['tokens']).values
# Train the Word2Vec Model
w2v_model1 = Word2Vec(stemmed_tokens, min_count = min_count, size = size, workers = workers, window = window, sg = sg)
print("Time taken to train word2vec model: " + str(time.time() - start_time))
w2v_model1.save(word2vec_model_file1)


Time taken to train word2vec model: 21.293802499771118


In [22]:
import numpy as np

# Load the model from the model file
sg_w2v_model1 = Word2Vec.load(word2vec_model_file1)

# Unique ID of the word
print("Index of the word 'min':")
print(sg_w2v_model1.wv.vocab["min"].index)

# Total number of the words 
print(len(sg_w2v_model1.wv.vocab))


Index of the word 'min':
231
37558


In [25]:
# Store the vectors for train data in following file
word2vec_filename = OUTPUT_FOLDER + 'train_review_word2vec.csv'
with open(word2vec_filename, 'w+') as word2vec_file:
    for index, row in enumerate(data['tokens'].tolist()):
        model_vector = (np.mean([sg_w2v_model1[token] for token in row], axis=0)).tolist()
        if index == 0:
            header = ",".join(str(ele) for ele in range(100))
            word2vec_file.write(header)
            word2vec_file.write("\n")
        # Check if the line exists else it is vector of zeros
        if type(model_vector) is list:  
            line1 = ",".join( [str(vector_element) for vector_element in model_vector] )
        else:
            line1 = ",".join([str(0) for i in range(100)])
        word2vec_file.write(line1)
        word2vec_file.write('\n')


C:\Users\LENOVO\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


# Storing trained vectors 

In [26]:
X_train_vectors=pd.read_csv('train_review_word2vec.csv')

# Target variable

In [27]:
y = data.Rating ; type(y) ; y.shape

(14343,)

# Importing Test dataset

In [28]:
data_test = pd.read_csv("C:\\Users\\LENOVO\\Desktop\\test.csv")

In [29]:
data_test.shape
data_test.head(3)
# 6148 and 2 

ID                                             Review
0   0  just superb rendezvous just perfect property s...
1   1  better close staten island ferry easy subway, ...
2   2  enjoyed stay, just come long weekend barcelona...

# Preprocessing Test Data

In [30]:
# Replacing contractions (shorter vesions of words) with full word (n't , 'nt  as not)
for index, row in data_test.iterrows():
    phrase = row['Review']
    phrase = phrase.lower() #Convert to lowercase
    phrase = re.sub(r"n\'t", " not", phrase) # replace n't  with not
    phrase = re.sub(r"\'nt", " not", phrase) # replace 'nt  with not 
    data_test.at[index, 'Review_2'] = phrase
data_test[0:3]

ID                                             Review  \
0   0  just superb rendezvous just perfect property s...   
1   1  better close staten island ferry easy subway, ...   
2   2  enjoyed stay, just come long weekend barcelona...   

                                            Review_2  
0  just superb rendezvous just perfect property s...  
1  better close staten island ferry easy subway, ...  
2  enjoyed stay, just come long weekend barcelona...

In [31]:
# TEXT NORMALIZATION   or PRE-PROCESSING

##Creating a list of stop words and adding custom stopwords
stop_words = set(stopwords.words("english")) -set(["not"])
##Creating a list of custom stopwords
new_words = ['n',"th","hotel",'hotels','room','rooms','restaurant','restaurants','e','ca','nd','wo','el','etc']
stop_words = stop_words.union(new_words)

for index, row in data_test.iterrows():
    text = re.sub('[^a-zA-Z]', ' ', row['Review_2']) # Remove punctuations    
    # text = text.lower() #Convert to lowercase
    text = re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)  # remove tags
    text = re.sub("(\\d|\\W)+"," ",text)  # remove special characters and digits
    text = text.split()  # Convert to list from string
    
    #Lemmatisation
    lemmatizer = WordNetLemmatizer()
    text = [lemmatizer.lemmatize(word) for word in text if not word  in stop_words] 
    text_string = " ".join(text)
    data_test.at[index, 'Clean_review'] = text_string
data_test.head(3)

ID                                             Review  \
0   0  just superb rendezvous just perfect property s...   
1   1  better close staten island ferry easy subway, ...   
2   2  enjoyed stay, just come long weekend barcelona...   

                                            Review_2  \
0  just superb rendezvous just perfect property s...   
1  better close staten island ferry easy subway, ...   
2  enjoyed stay, just come long weekend barcelona...   

                                        Clean_review  
0  superb rendezvous perfect property singapore f...  
1  better close staten island ferry easy subway f...  
2  enjoyed stay come long weekend barcelona staye...

In [32]:
data_test.drop(['Review_2'], axis=1,inplace=True)

In [35]:
data_test['tokens'] = data_test["Clean_review"].apply(word_tokenize)

In [36]:
data_test.head()

ID                                             Review  \
0   0  just superb rendezvous just perfect property s...   
1   1  better close staten island ferry easy subway, ...   
2   2  enjoyed stay, just come long weekend barcelona...   
3   3  muse great, muse hotel great, did n't hear noi...   
4   4  conveniently located morning flight, family st...   

                                        Clean_review  \
0  superb rendezvous perfect property singapore f...   
1  better close staten island ferry easy subway f...   
2  enjoyed stay come long weekend barcelona staye...   
3  muse great muse great not hear noise reviewer ...   
4  conveniently located morning flight family sta...   

                                              tokens  
0  [superb, rendezvous, perfect, property, singap...  
1  [better, close, staten, island, ferry, easy, s...  
2  [enjoyed, stay, come, long, weekend, barcelona...  
3  [muse, great, muse, great, not, hear, noise, r...  
4  [conveniently, located, morning, flight, famil...

# Word2vec for test dataset

In [40]:
from gensim.models import Word2Vec
import time

size = 100
window = 3
min_count = 1
workers = 3
sg = 1

OUTPUT_FOLDER=""

word2vec_model_file2 = OUTPUT_FOLDER + 'word2vec_' + str(size) + '.model'
start_time = time.time()
stemmed_tokens = pd.Series(data_test['tokens']).values
# Train the Word2Vec Model
w2v_model2 = Word2Vec(stemmed_tokens, min_count = min_count, size = size, workers = workers, window = window, sg = sg)
print("Time taken to train word2vec model: " + str(time.time() - start_time))
w2v_model2.save(word2vec_model_file2)


Time taken to train word2vec model: 8.69957709312439


In [41]:
import numpy as np

# Load the model from the model file
sg_w2v_model2 = Word2Vec.load(word2vec_model_file2)

# Unique ID of the word
print("Index of the word 'min':")
print(sg_w2v_model2.wv.vocab["min"].index)

# Total number of the words 
print(len(sg_w2v_model2.wv.vocab))


Index of the word 'min':
190
23896


In [42]:
# Store the vectors for train data in following file
word2vec_filename = OUTPUT_FOLDER + 'test_review_word2vec.csv'
with open(word2vec_filename, 'w+') as word2vec_file:
    for index, row in enumerate(data_test['tokens'].tolist()):
        model_vector = (np.mean([sg_w2v_model2[token] for token in row], axis=0)).tolist()
        if index == 0:
            header = ",".join(str(ele) for ele in range(100))
            word2vec_file.write(header)
            word2vec_file.write("\n")
        # Check if the line exists else it is vector of zeros
        if type(model_vector) is list:  
            line1 = ",".join( [str(vector_element) for vector_element in model_vector] )
        else:
            line1 = ",".join([str(0) for i in range(100)])
        word2vec_file.write(line1)
        word2vec_file.write('\n')


C:\Users\LENOVO\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


# Storing Test data vectors

In [43]:
X_test_vectors=pd.read_csv('test_review_word2vec.csv')

In [44]:
# # Splitting dataset in 80:20 ratio
# frpm previous models. found this ratio to give good results
X_train1,X_test1,y_train1,y_test1 = train_test_split(X_train_vectors,y,test_size=0.20,random_state=123,stratify=y)
X_train1.shape # (11474, 3000)
X_test1.shape # (2869, 3000)
y.shape  # 14343 reviews

(14343,)

In [45]:
from xgboost import XGBClassifier

model = XGBClassifier(max_depth=10,random_state=1,learning_rate=0.05,seed=1)
model.fit(X_train1, y_train1)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.05, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=1, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, seed=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [48]:
# for training data
y_pred_train = model.predict(X_train1)

print("Accuracy score on train data:" ,accuracy_score(y_train1,y_pred_train))
print("F1 macro score on train data:",f1_score(y_train1,y_pred_train, average='macro'))
print(classification_report(y_train1,y_pred_train))
print(confusion_matrix(y_train1,y_pred_train))
# plot_confusion_matrix(model_log, X_train1,y_train1,cmap=plt.cm.Blues); plt.show();

# for test data 
y_pred_test = model.predict(X_test1)

print("Accuracy score on test data:" ,accuracy_score(y_test1,y_pred_test))
print("F1 macro score on test data:",f1_score(y_test1,y_pred_test, average='macro'))
print(classification_report(y_test1,y_pred_test))
print(confusion_matrix(y_test1,y_pred_test))

# plotting confusion matrix
# plot_confusion_matrix(model_log, X_test1,y_test1,cmap=plt.cm.Blues); plt.show();

Accuracy score on train data: 0.9992156179187729
F1 macro score on train data: 0.999376588941433
              precision    recall  f1-score   support

           1       1.00      1.00      1.00       782
           2       1.00      1.00      1.00       998
           3       1.00      1.00      1.00      1208
           4       1.00      1.00      1.00      3337
           5       1.00      1.00      1.00      5149

    accuracy                           1.00     11474
   macro avg       1.00      1.00      1.00     11474
weighted avg       1.00      1.00      1.00     11474

[[ 781    0    0    1    0]
 [   0  997    0    0    1]
 [   0    0 1208    0    0]
 [   0    0    0 3330    7]
 [   0    0    0    0 5149]]
Accuracy score on test data: 0.5845242244684559
F1 macro score on test data: 0.47858605739152804
              precision    recall  f1-score   support

           1       0.64      0.60      0.62       195
           2       0.41      0.35      0.38       250
           3 

In [49]:

model = XGBClassifier(max_depth=10,random_state=1,learning_rate=0.05,seed=1)
model.fit(X_train_vectors, y)
# for training data
y_pred_train = model.predict(X_train_vectors)

print("Accuracy score :" ,accuracy_score(y,y_pred_train))
print("F1 macro score :",f1_score(y,y_pred_train, average='macro'))
print(classification_report(y,y_pred_train))
print(confusion_matrix(y,y_pred_train))

# for test data 
test_pred = model.predict(X_test_vectors)


Accuracy score : 0.9988844732622185
F1 macro score : 0.9991599040393275
              precision    recall  f1-score   support

           1       1.00      1.00      1.00       977
           2       1.00      1.00      1.00      1248
           3       1.00      1.00      1.00      1510
           4       1.00      1.00      1.00      4172
           5       1.00      1.00      1.00      6436

    accuracy                           1.00     14343
   macro avg       1.00      1.00      1.00     14343
weighted avg       1.00      1.00      1.00     14343

[[ 976    0    0    1    0]
 [   0 1246    0    1    1]
 [   0    0 1510    0    0]
 [   0    0    0 4159   13]
 [   0    0    0    0 6436]]


In [50]:
type(test_pred)
test_pred.shape
test_pred[0:5]

# converting pred values to df

df_pred = pd.DataFrame(test_pred, columns=['Rating'])
df_pred.shape

data_test.columns
df_pred["ID"] = data_test["ID"]

col_titles = ["ID","Rating"]
df_pred = df_pred.reindex(columns=col_titles)
df_pred.head()

ID  Rating
0   0       5
1   1       2
2   2       1
3   3       5
4   4       4

In [51]:
# Exporting to csv file
df_pred.to_csv(r'C:\Users\LENOVO\Desktop\XGB.csv',index=False)